# Enzyme Classification Using Protein Structure Data

## Introduction

This project utilizes the **RCSB_PDB Human Macromolecular Structure Dataset** from **Kaggle.com**. The dataset contains **11,832 protein structures** obtained from the **RCSB Protein Data Bank (PDB)** between **2015 and 2023**. It includes **51 features** spanning taxonomic data, sequence characteristics, crystallization conditions, and more — offering rich potential for machine learning tasks, such as **enzyme classification**.

### Problem Statement

The goal of this project is to build a model that predicts the **enzyme classification** of a given protein structure based on selected features. Understanding an enzyme’s class can reveal its biological function, potential behavior in the body, and impact on health, which is critical in **drug discovery** and **healthcare applications**.

### Features Used

Out of the 51 features available, **17 were selected** based on their biological relevance and potential impact on enzyme classification:

- **Structure and Sequence**:
  - Number of amino acids (residues)
  - Number of chains
  - Amino acid sequence
  - Number of helices, sheets, and coils (secondary structure)

- **Assembly Information**:
  - Oligomeric state
  - Stoichiometry

- **Molecular Properties**:
  - Molecular weight
  - Macromolecule type

- **Crystallization Conditions**:
  - pH
  - Temperature
  - Percent solvent content
  - Crystallization method

These features were chosen because enzymes of the same class often exhibit **similar structural motifs and environmental conditions**. By learning these patterns, a model can **infer the class of an enzyme** with high accuracy.

The **target variable** is the **enzyme classification**, which is represented as a categorical numerical code in the dataset.

## Data Pre-processing

In this section, we:
- Load the dataset and examine its structure.
- Clean the data by handling missing values.
- Encode categorical variables.
- Extract the target variable and reduce the enzyme classification to its first digit.
- Normalize numerical features using standard scaling.
- Split the data into training and testing sets for model evaluation.


In [3]:
import seaborn as sns
import pandas as pd

In [ ]:
path = "RCSB_PDB_Macromolecular_Structure_Dataset_with_Structural_Features.csv"
Dataset = pd.read_csv(path)

In [ ]:
num_columns = Dataset.shape[1]
print("Number of columns:",num_columns)

Number of columns: 51


In [6]:
df = pd.DataFrame()
columns = ['PDB ID', 'Percent Solvent Content', 'Crystallization Method', 'pH', 'Temp (K)', 'Molecular Weight per Deposited Model', 'Sequence', 'Entity Macromolecule Type', 'Molecular Weight (Entity)', 'EC Number', 'Oligomeric State', 'Stoichiometry', 'Number of Residues', 'Number of Chains', 'Helix', 'Sheet', 'Coil']
selected_columns = Dataset[columns]

# Adding the selected columns to the empty DataFrame
df = pd.concat([df, selected_columns], axis=1)

print(df.head)

<bound method NDFrame.head of       PDB ID  Percent Solvent Content         Crystallization Method   pH  \
0       6SJZ                    44.16  VAPOR DIFFUSION, HANGING DROP  5.5   
1       5C37                    45.64  VAPOR DIFFUSION, HANGING DROP  8.0   
2       6NNA                    45.48  VAPOR DIFFUSION, HANGING DROP  7.6   
3       7M4C                    56.57  VAPOR DIFFUSION, SITTING DROP  7.5   
4       7M4E                    56.66  VAPOR DIFFUSION, SITTING DROP  7.5   
...      ...                      ...                            ...  ...   
11827   6T3M                    52.68  VAPOR DIFFUSION, HANGING DROP  7.5   
11828   6WLX                    42.21  VAPOR DIFFUSION, HANGING DROP  7.5   
11829   8D6J                    42.67  VAPOR DIFFUSION, HANGING DROP  NaN   
11830   7C6B                    52.74  VAPOR DIFFUSION, SITTING DROP  8.0   
11831   7D7U                    52.82  VAPOR DIFFUSION, SITTING DROP  8.0   

       Temp (K)  Molecular Weight per Deposit

In [7]:
sns.pairplot(data=df,
             x_vars=['PDB ID', 'Percent Solvent Content', 'Crystallization Method', 'pH', 'Temp (K)', 'Molecular Weight per Deposited Model', 'Sequence', 'Entity Macromolecule Type', 'Molecular Weight (Entity)', 'Oligomeric State', 'Stoichiometry', 'Number of Residues', 'Number of Chains', 'Helix', 'Sheet', 'Coil'],
             y_vars=['EC Number'],
             hue='EC Number',height = 2)

c:\Users\Calvi\anaconda3\Lib\site-packages\seaborn\axisgrid.py:123: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations
  self._figure.tight_layout(*args, **kwargs)


In [ ]:
#Set X and y 
X_df = df.drop('EC Number', axis=1)
y_df = df['EC Number']
X_df.head()

,PDB ID,Percent Solvent Content,Crystallization Method,pH,Temp (K),Molecular Weight per Deposited Model,Sequence,Entity Macromolecule Type,Molecular Weight (Entity),Oligomeric State,Stoichiometry,Number of Residues,Number of Chains,Helix,Sheet,Coil
0,6SJZ,44.16,"VAPOR DIFFUSION, HANGING DROP",5.5,293.0,97.20,GGSEFSVGQGPAKTMEEASKRSYQFWDTQPVPKLGEVVNTHGPVEP...,polypeptide(L),46.465,Hetero 2-mer,"A1, B1",1401.0,4.0,213.0,220.0,362.0
1,5C37,45.64,"VAPOR DIFFUSION, HANGING DROP",8.0,298.0,146.72,EQQVPILEKFCFTPHTEEGCLSERAALQEELQLCKGLVQALQTKVT...,polypeptide(L),72.049,Monomer,A1,1499.0,2.0,506.0,280.0,470.0
2,6NNA,45.48,"VAPOR DIFFUSION, HANGING DROP",7.6,291.0,147.01,QQVPILEKFCFTPHTEEGCLSERAALQEELQLCKGLVQALQTKVTQ...,polypeptide(L),71.965,Monomer,A1,1827.0,2.0,483.0,281.0,452.0
3,7M4C,56.57,"VAPOR DIFFUSION, SITTING DROP",7.5,277.0,44.09,AQPSSQKATNHNLHITEKLEVLAKAYSVQGDKWRALGYAKAINALK...,polypeptide(L),36.704,Monomer,A1,593.0,4.0,158.0,48.0,114.0
4,7M4E,56.66,"VAPOR DIFFUSION, SITTING DROP",7.5,277.0,44.19,AQPSSQKATNHNLHITEKLEVLAKAYSVQGDKWRALGYAKAINALK...,polypeptide(L),36.736,Monomer,A1,463.0,4.0,156.0,41.0,113.0


In [ ]:
# Checking for missing data
print(df.isnull().sum())

PDB ID                                     0
Percent Solvent Content                  131
Crystallization Method                   297
pH                                      4049
Temp (K)                                 316
Molecular Weight per Deposited Model       0
Sequence                                   0
Entity Macromolecule Type                  0
Molecular Weight (Entity)                  0
EC Number                                674
Oligomeric State                           0
Stoichiometry                              0
Number of Residues                       392
Number of Chains                         392
Helix                                    392
Sheet                                    392
Coil                                     392
dtype: int64


In [ ]:
# Checking size of data
print(X_df.shape)
print(y_df.shape)

(11832, 16)
(11832,)


## Model Setup

In this section, we set up one or more machine learning models to perform enzyme classification. We will start with a baseline model (e.g., Random Forest) and may explore more advanced options (e.g., neural networks) depending on performance.

Key objectives:
- Train the model using the training data.
- Evaluate initial performance using accuracy and classification reports.


## Hyperparameter Tuning

Here, we tune model parameters to improve performance.

We'll explore:
- Different learning rates, optimizers, and architectures (for neural nets)
- Tree depth, number of estimators, and splitting criteria (for tree-based models)
- Cross-validation to validate model generalization


## Results

This section presents the model's performance on the test data.

We evaluate using:
- Accuracy
- Confusion Matrix
- Classification Report

Visualizations and metric comparisons will also be included to interpret the results better.


## Discussion

### What Worked
- Effective feature selection significantly improved prediction accuracy.
- Certain models (e.g., Random Forest or Neural Network) may generalize well.

### What Didn’t
- Some enzyme classes were underrepresented, which may have affected performance.
- Class imbalance may have skewed accuracy in favor of dominant classes.

### Future Work
- Try deep learning models or transformer-based sequence models.
- Perform feature engineering on sequence data (e.g., embeddings, k-mers).
- Use data augmentation to balance classes.
